In [6]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("deepseek.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("DEEPSEEK_API_KEY")
api_base = os.getenv("DEEPSEEK_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

# ChatDOc:和文件聊天

In [5]:
#导入必须的包
from langchain_community.document_loaders import UnstructuredExcelLoader,Docx2txtLoader,PyPDFLoader
from langchain_text_splitters import  CharacterTextSplitter
from langchain_community.embeddings import  HuggingFaceEmbeddings
from langchain_community.vectorstores import  Chroma
#导入聊天所需的模块
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

import os



# 为 DeepSeek 配置（可选）
DEEPSEEK_CONFIG = {
    #"model": "gpt-3.5-turbo",  # 或者使用 DeepSeek 兼容模型
    #"openai_api_key": os.environ["OPENAI_API_KEY"],
    # 如果使用 DeepSeek:
    "model": "deepseek-chat",
    "openai_api_base": os.environ["DEEPSEEK_API_BASE"],
    "openai_api_key": os.environ["DEEPSEEK_API_KEY"]
}

#定义chatdoc
class ChatDoc():
    def __init__(self):
        self.doc = None
        self.splitText = [] #分割后的文本
        self.template = [
            ("system","你是一个处理文档的秘书,你从不说自己是一个大模型或者AI助手,你会根据下面提供的上下文内容来继续回答问题.\n 上下文内容\n {context} \n"),
            ("human","你好！"),
            ("ai","你好"),
            ("human","{question}"),
        ]
        self.prompt = ChatPromptTemplate.from_messages(self.template)

    def getFile(self):
        doc = self.doc
        loaders = {
            "docx":Docx2txtLoader,
            "pdf":PyPDFLoader,
            "xlsx":UnstructuredExcelLoader,
        }
        file_extension = doc.split(".")[-1]
        loader_class = loaders.get(file_extension)
        if loader_class:
            try:
                loader = loader_class(doc)
                text = loader.load()
                return text
            except Exception as e: 
                print(f"Error loading {file_extension} files:{e}") 
        else:
             print(f"Unsupported file extension: {file_extension}")
             return  None 

    #处理文档的函数
    def splitSentences(self):
        full_text = self.getFile() #获取文档内容
        if full_text != None:
            #对文档进行分割
            text_split = CharacterTextSplitter(
                chunk_size=150,
                chunk_overlap=20,
            )
            texts = text_split.split_documents(full_text)
            self.splitText = texts
    
    #向量化与向量存储
    def embeddingAndVectorDB(self):
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        return Chroma.from_documents(
            documents=self.splitText,
            embedding=embeddings,
        )
        
        # return db
    
    #提问并找到相关的文本块
    def askAndFindFiles(self,question):
        db = self.embeddingAndVectorDB()
        #retriever = db.as_retriever(search_type="mmr")
        retriever = db.as_retriever(search_type="similarity_score_threshold",search_kwargs={"score_threshold":.5,"k":1})
        return retriever.get_relevant_documents(query=question)
    
    #用自然语言和文档聊天
    def chatWithDoc(self,question):
        _content = ""
        context = self.askAndFindFiles(question)
        for i in context:
            _content += i.page_content
        
        messages = self.prompt.format_messages(context=_content,question=question)
        chat = ChatOpenAI(
            temperature=0,
            **DEEPSEEK_CONFIG
        )
        return chat.invoke(messages)

chat_doc = ChatDoc()
chat_doc.doc = "example/fake.docx"
chat_doc.splitSentences()
chat_doc.chatWithDoc("公司在哪儿？")  # 替换为实际问题


AIMessage(content='根据提供的上下文内容，宏图科技发展有限公司的具体地理位置信息并未提及。建议您查阅公司公开的工商注册信息、官方网站或通过企业信用信息公示系统查询，以获取准确的注册地址和办公地点信息。  \n\n若您需要进一步了解该公司的资产状况或收购相关事宜，可参考结论与建议部分提及的尽职调查和风险评估要点。', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 128, 'total_tokens': 203, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_cache_miss_tokens': 64}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'finish_reason': 'stop', 'logprobs': None}, id='run--4020fc65-2a0c-44f2-b191-1dc09d5a6b06-0')